In [24]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense 
from sklearn.metrics import roc_curve
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from scikeras.wrappers import KerasClassifier

import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import auc
import numpy as np
from imblearn.over_sampling import SMOTE


#array de colunas


In [25]:
weeks1 = pd.read_csv('../data_raw/dados/2016-1/tabela processada nota media mediana iteracao.csv')
features = ['Aprovado']

for i in range(18):
  features.append(f'Week{i}')
  features.append(f'MedianaW{i}')
  features.append(f'MediaW{i}')
how = ['first']

for i in range(18*3):
  how.append(f'last')

dict_agg = dict(zip(features,how))

weeks2_semnotas = pd.read_csv('../data_raw/dados/2016-2/tabela final .csv')
weeks2_comnotas = pd.read_csv('../data_raw/dados/2016-2/tabela de iteracoes com nota add.csv')
#weeks2 = weeks2_semnotas.join(weeks2_comnotas,on='Nome',how='inner')
weeks2_raw = pd.concat([weeks2_semnotas, weeks2_comnotas])
weeks2 = weeks2_raw.groupby(weeks2_raw['Nome']).agg(dict_agg)
weeks2.dropna(inplace=True)
weeks3 = pd.read_csv('../data_raw/dados/2017-1/tabela final.csv')
weeks4 = pd.read_csv('../data_raw/dados/2017-2/tabela com media mediana e interacao e nota.csv')

weeks_raw =  [weeks1, weeks2, weeks3, weeks4]


In [26]:
def weeks_formating(weeks):
  
  weeks.fillna(0, inplace=True)
  weeks.drop(['Unnamed: 0.1','Notas', 'Unnamed: 0', 'Nome'], inplace=True, axis=1, errors='ignore')
  weeks['Aprovado'] = pd.Categorical(weeks['Aprovado'])
  weeks['Aprovado'] = weeks.Aprovado.cat.codes
  weeks = (weeks
          .replace('\$\s+','', regex=True)
          .astype(float)
          .applymap('{:.2f}'.format))
  weeks = weeks.astype(float)
  weeks_lbl = weeks['Aprovado']
  return weeks, weeks_lbl

In [27]:
def oversample_data(X_train, y_train):
  smote = SMOTE(random_state = 101)
  X_oversample, y_oversample = smote.fit_resample(X_train, y_train)
  return X_oversample, y_oversample

In [28]:
def weeks_data(data, weeks_num):

  features = []

  for i in range(weeks_num):
    features.append(f'Week{i}')
    features.append(f'MedianaW{i}')
    features.append(f'MediaW{i}')
    
  features.append(f'Aprovado')
  features.remove('Aprovado')
  weeks = data[features]
  weeks = weeks.to_numpy()
  return weeks

In [29]:
def build_model(weeks_num):
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model = Sequential()
    model.add(LSTM(30, input_shape=(weeks_num, 3)))
    model.add(Dense(1, activation='sigmoid'))
    #model.add(Softmax(1))
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=["accuracy"])
    return model


In [30]:
weeks_num = 15

for week in weeks_raw:
  weeks, label = weeks_formating(week)

  
  for i in range(weeks_num):

    data = weeks_data(weeks, i+1)

    X, y = data, label

    loo = LeaveOneOut()

    X = X.reshape(len(X), i+1, 3)

    # model = KerasClassifier(model=build_model(i+1), epochs=5, batch_size=3, verbose=0)
    # dentro do for: clear session, complie model, fit no kerasclassifier, usar o predict_proba(), agregar em yhat
    # fora do for: usar a func dada, jogar na auc
    # scores = cross_validate(model, X, y, scoring='neg_mean_absolute_error', cv=loo, n_jobs=-1)

    y_true, y_pred = list(), list()
    for train_ix, test_ix in loo.split(X):
      	# split data
        X_train, X_test = X[train_ix, :], X[test_ix, :]
        y_train, y_test = y[train_ix], y[test_ix]
        # fit model
        model = build_model(i+1)
        model.fit(X_train, y_train)
        # evaluate model
        tf.keras.backend.clear_session()
        yhat = model.predict(X_test, verbose=0)
        # store
        y_true.append(y_test.iloc[0])
        y_pred.append(yhat[0])

    fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_true,y_pred)
    auc_keras = auc(fpr_keras, tpr_keras)

    plt.plot(fpr_keras, tpr_keras, label='Week {} (area = {:.3f})'.format(i+1, auc_keras))

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    
  plt.title('ROC curve 15 weeks')
  plt.legend(loc='best')
  plt.show()
    


1/1 [==============================] - 6s 6s/step - loss: 0.6713 - accuracy: 0.3929
1.0
1/1 [==============================] - 6s 6s/step - loss: 0.6933 - accuracy: 0.3571
1.0
1/1 [==============================] - 7s 7s/step - loss: 0.6876 - accuracy: 0.3214
1.0
1/1 [==============================] - 4s 4s/step - loss: 0.7066 - accuracy: 0.2500
0.0
1/1 [==============================] - 5s 5s/step - loss: 0.6856 - accuracy: 0.3929
1.0
1/1 [==============================] - 5s 5s/step - loss: 0.8335 - accuracy: 0.2500
1.0
1/1 [==============================] - 5s 5s/step - loss: 0.7296 - accuracy: 0.2143
0.0
1/1 [==============================] - 7s 7s/step - loss: 0.6885 - accuracy: 0.2857
1.0
1/1 [==============================] - 5s 5s/step - loss: 0.6878 - accuracy: 0.3929
1.0
1/1 [==============================] - 5s 5s/step - loss: 0.7264 - accuracy: 0.2500
0.0
1/1 [==============================] - 4s 4s/step - loss: 0.7483 - accuracy: 0.2143
0.0
1/1 [===========================